In [2]:
%cd GeDi

d:\d-project\detox\GeDi


In [3]:
from transformers import pipeline
from datasets import load_dataset
import jsonlines
from tqdm.autonotebook  import tqdm
import numpy as np
import torch
from modeling_gpt2 import GPT2LMHeadModel

from transformers import (
    GPT2Config,
    GPT2Tokenizer
)

In [4]:
data_name = "SetFit/bbc-news"
dataset = load_dataset(data_name, split="train")

Using custom data configuration SetFit--bbc-news-dbeb222bfdd4d6ed
Found cached dataset json (C:/Users/heegyukim/.cache/huggingface/datasets/SetFit___json/SetFit--bbc-news-dbeb222bfdd4d6ed/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [6]:
mode = "detoxifier"
code_desired = "dirty"
code_undesired = "clean"
model_type = 'gpt2'
gen_type = "gedi"
gen_model_name_or_path = "gpt2" # "gpt2-medium"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),}
config_class, model_class, tokenizer_class = MODEL_CLASSES["gpt2"]
tokenizer = tokenizer_class.from_pretrained(gen_model_name_or_path, do_lower_case=False)

model = model_class.from_pretrained(gen_model_name_or_path)#, load_in_half_prec=True)
model = model.to(device)

gedi_model_name_or_path = 'gedi_detoxifier'
gedi_model = model_class.from_pretrained(gedi_model_name_or_path).to(device)

no logit scale initialized for gpt2


In [7]:
#setting arguments for generation
#max generation length
gen_length = 200
#omega from paper, higher disc_weight means more aggressive topic steering (30)
disc_weight = 30
#1 - rho from paper, should be between 0 and 1 higher filter_p means more aggressive topic steering
filter_p = 0.8
#tau from paper, preserves tokens that are classified as correct topic
target_p = 0.8
#hyperparameter that determines class prior, set to uniform by default
class_bias = 0

if gen_length>1024:
  length = 1024
else:
  length = gen_length

In [8]:
gen_length = 400
def generate_text(prompt):
  text_ids = tokenizer.encode(prompt)
  encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0).to(device)
  if encoded_prompts.shape[1] > 512:
    encoded_prompts = encoded_prompts[:, -512:]

  # multi_code = tokenizer.encode(secondary_code)
  attr_class = 1

  generated_sequence = model.generate(
    input_ids=encoded_prompts,
    pad_lens=None,
    max_length=min(1024, encoded_prompts.shape[1] + gen_length),
    min_length=min(1024, encoded_prompts.shape[1] + gen_length),
    top_k=None,
    top_p=1.0,
    repetition_penalty= 1.2,
    rep_penalty_scale= 10,
    eos_token_ids = tokenizer.eos_token_id,
    pad_token_id = 0,
    do_sample= True,
    penalize_cond= True,
    gedi_model= gedi_model,
    tokenizer= tokenizer,
    disc_weight= disc_weight,
    filter_p = filter_p,
    target_p = target_p,
    class_bias = class_bias,
    attr_class = attr_class,
    code_0 = code_desired,
    code_1 = code_undesired,
    multi_code=None,
    num_return_sequences=5
    )

  texts = [tokenizer.decode(output, skip_special_tokens=True)[len(prompt):] for output in generated_sequence.tolist()[0]]
  return texts

prompt = "It is really "
generate_text(prompt)

["only Boulder as a nuclear power reactor which comes to mind,\n\nthis shows that people don't understand it at all. When I was far away between Alaska and Idaho my assignment centers were sunny on sunshine days (and back roads not). Living there grew up in suburban homes; if you weren't able to rouse or vent your fears about pollution, the community would have shrapnel dropped thousands of feet from shores like 70 stories of guttering wood used by sailors for vibrating hair removal and grease treatment. Just one friend who had similar problems overheard his parents digging dirt out an embankment when he got out; he forgot their attendance laxity for ever filling these private spaces with heavy weeds once the place looked more warm from outside across a 41 mile stretch. Eventually 71 was available—only 40 miles east of Boulder as pictured above, but standing 200 yards apart along with 14 clams—with Rangers together guiding the gleaming powder hole through field after field in this way 

In [ ]:
filename = data_name.replace("/", "__")
with jsonlines.open(f"data/gedi-small-bbc-400-omega60-rp10.jsonl", 'w') as f:
    for item in tqdm(dataset, leave=False):
        text = item['text'].strip()
        # if len(text) >= 1024:
        #     text = text[-1024:]
        gens = generate_text(text)
        # gens = [g['generated_text'][len(text):] for g in gens]
        gens = [g[len(text):] for g in gens]
        item['generation'] = gens
        # item['prediction'] = classifier(gens)
        f.write(item)
        # print(text)
        # print(item['label'])
        # print(gens)
        # print(item['prediction'])
        # break

100%|█████████▉| 1223/1225 [5:59:56<00:32, 16.44s/it]  Token indices sequence length is longer than the specified maximum sequence length for this model (1141 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
import jsonlines
from collections import defaultdict

label2id = {
  "business": 0,
  "entertainment": 1,
  "politics": 2,
  "sport": 3,
  "tech": 4
}
fix_label = {
  2: 3,
  4: 2,
  3: 1,
  1: 0,
  0: 4
}
total = defaultdict(lambda: 0)
correct = defaultdict(lambda: 0)

with jsonlines.open("data/gedi-small-bbc-400-omega60.jsonl") as f:
  for i, item in tqdm(enumerate(f)):
    label = fix_label[item['label']]
    preds = classifier(item['generation'])
    # preds = item['prediction']

    total[label] += 5
    # print(label2id[p['label']], preds)
    # if i == 50:
    #   break

    for p in preds:
      if label2id[p['label']] == label:
        correct[label] += 1

    # break

for k in total.keys():
    print(k, correct[k] / total[k])

0it [00:00, ?it/s]


NameError: name 'classifier' is not defined

In [ ]:

for k in sorted(total.keys()):
    print(k, correct[k] / total[k])

0 0.6692307692307692
1 0.6180952380952381
2 0.4033057851239669
3 0.43854545454545457
4 0.5839622641509434


In [ ]:
correct

defaultdict(<function __main__.<lambda>()>,
            {3: 958, 0: 1169, 1: 839, 4: 678, 2: 712})